![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# Hardcore DL by Spark NLP

## Explain Documents with Deep Learning

In [1]:
import sys
import time

#Spark ML and SQL
from pyspark.ml import Pipeline, PipelineModel
from pyspark.sql.functions import array_contains
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
#Spark NLP
import sparknlp
from sparknlp.pretrained import PretrainedPipeline
from sparknlp.annotator import *
from sparknlp.common import RegexRule
from sparknlp.base import *

### Let's create a Spark Session for our app

Let's take a look at what's behind `sparknlp.start()` function:

In [2]:
spark = SparkSession.builder \
        .appName("Spark NLP") \
        .master("local[*]") \
        .config("spark.driver.memory", "6G") \
        .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
        .config("spark.jars.packages", "JohnSnowLabs:spark-nlp:2.0.1,com.johnsnowlabs.nlp:spark-nlp-ocr_2.11:2.0.1") \
        .config("spark.jars.repositories", "http://repo.spring.io/plugins-release") \
        .getOrCreate()

In [3]:
pipeline = PretrainedPipeline('explain_document_dl', lang='en')

CPU times: user 14.5 ms, sys: 9.73 ms, total: 24.2 ms
Wall time: 1min 9s


We simply send the text we want to transform and the pipeline does the work.

In [4]:
text = 'He would love to visit many beautful cities wth you. He lives in an amazing country.'
result = pipeline.annotate(text)

CPU times: user 39.9 ms, sys: 18.6 ms, total: 58.5 ms
Wall time: 3.17 s


We can see the output of each annotator below. This one is doing so many things at once!

In [5]:
list(result.keys())

['stem',
 'checked',
 'lemma',
 'document',
 'pos',
 'token',
 'ner_con',
 'ner',
 'embeddings',
 'sentence']

In [6]:
result['sentence']

['He would love to visit many beautful cities wth you.',
 'He lives in an amazing country.']

In [7]:
result['lemma']

['He',
 'would',
 'love',
 'to',
 'visit',
 'many',
 'beautful',
 'city',
 'wth',
 'you',
 '.',
 'He',
 'life',
 'in',
 'an',
 'amazing',
 'country',
 '.']

In [8]:
list(zip(result['checked'], result['pos']))

[('He', 'PRP'),
 ('Fould', 'MD'),
 ('love', 'VB'),
 ('to', 'TO'),
 ('list', 'VB'),
 ('many', 'JJ'),
 ('beautful', 'JJ'),
 ('ties', 'NNS'),
 ('with', 'NN'),
 ('you', 'PRP'),
 ('.', '.'),
 ('He', 'PRP'),
 ('live', 'VBZ'),
 ('in', 'IN'),
 ('an', 'DT'),
 ('main', 'JJ'),
 ('countrymen', 'NN'),
 ('.', '.')]

In [9]:
list(zip(result['checked'], result['pos']))

[('He', 'PRP'),
 ('Fould', 'MD'),
 ('love', 'VB'),
 ('to', 'TO'),
 ('list', 'VB'),
 ('many', 'JJ'),
 ('beautful', 'JJ'),
 ('ties', 'NNS'),
 ('with', 'NN'),
 ('you', 'PRP'),
 ('.', '.'),
 ('He', 'PRP'),
 ('live', 'VBZ'),
 ('in', 'IN'),
 ('an', 'DT'),
 ('main', 'JJ'),
 ('countrymen', 'NN'),
 ('.', '.')]

### Now let's try to use this pipleine to explain a PDF file

In [10]:
from sparknlp.ocr import OcrHelper
data = OcrHelper().createDataset(spark, './immortal_text.pdf')
data.show()

+--------------------+--------------------+-------+------+
|                text|            filename|pagenum|method|
+--------------------+--------------------+-------+------+
|would have been a...|file:/Users/maziy...|      0|  text|
+--------------------+--------------------+-------+------+

CPU times: user 3.91 ms, sys: 4.25 ms, total: 8.16 ms
Wall time: 5.1 s


We can see the output of each annotator below.

In [38]:
pipeline.transform(data).select("token.result", "pos.result").show()

+--------------------+--------------------+
|              result|              result|
+--------------------+--------------------+
|[would, have, bee...|[MD, VB, VBN, DT,...|
+--------------------+--------------------+

